In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

branch4 = pd.read_excel("pronostico_demanda/excel_branches/branch4.xlsx").drop(columns=['Unnamed: 0'])


In [20]:
branch4

,branch,semana,item_id,quantity
0,4,108,2,1
1,4,109,2,0
2,4,110,2,0
3,4,111,2,0
4,4,112,2,0
...,...,...,...,...
38411,4,194,2283,0
38412,4,195,2283,0
38413,4,196,2283,0
38414,4,197,2283,0


In [21]:
dicc_branch4 = dict()
branch = branch4.groupby(['item_id', 'semana'], as_index=False).agg({'quantity': 'sum'})

val_unicos = branch['item_id'].unique().tolist()

for v in range(len(val_unicos)):
    item = val_unicos[v]
    b_it = branch.loc[branch['item_id']==item]
    dicc_branch4[v] = b_it

In [22]:
# Diferencia binaria

def dif_binaria(y_true, predictions):
    pass

In [23]:
# MAE

# https://datagy.io/mae-python/

import numpy as np

def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))


In [24]:
# ------ FUNCIÓN MÉTODO PROMEDIO MÓVIL SIMPLE ------

import math
import datetime 
#window: 4. utiliza 4 semanas anteriores para el cálculo del promedio

#branch_item: dataset con los datos del item x para la branch y


datos_pms_branches = pd.DataFrame()
datos_pms_branches['branch'] = None
datos_pms_branches['item'] = None
datos_pms_branches['mae'] = None

def promedio_movil_simple(dicc_branch, item_key):

    data = dicc_branch[item_key]
    item = data['item_id'].iloc[0]
    
    data2 = data.copy()
    data = data2.drop(columns=['item_id'])
    
    #datos: [semana, quantity]
    
    data_training = data[:math.floor(int((len(data)*2/3)))+1]
    data_test = data[-math.ceil((int(len(data)*1/3))):]
    
    # Se calcula el promedio movil simple para los datos de entrenamiento
    data_training['MA'] = data_training['quantity'].rolling(window=4).mean().shift(1)

    #creo el dataset que tendrá las predicciones + la anterior, utilizando como dato real el del data_set y no la predicción 
    # anterior hecha
    data_forecast = data_training.copy()

    
    #producir pronóstico 
    for i in range(len(data_test)):
        #1. creo otra fila en data_forecast que contenga una semana más, 0(cantidad) y 0(promedio).
        data_forecast.loc[len(data_forecast)]=[data_forecast.iloc[len(data_forecast)-1][0] + 1, 0, 0]
        
        #1. para la columna MA recalculo el promedio y ya no es 0
        data_forecast['MA'] = data_forecast['quantity'].rolling(window=4).mean().shift(1)
        
        #1. reemplaza como valor real histórico el pronóstico para calcular la siguiente predicción con ese dato
        data_forecast['quantity'].loc[len(data_forecast)-1] = data_forecast['MA'].loc[len(data_forecast)-1]
        
        #2. creo otra fila en data_forecast que contenga una semana más, 0(cantidad) y 0(promedio).
        data_forecast.loc[len(data_forecast)]=[data_forecast.iloc[len(data_forecast)-1][0] + 1, 0, 0]
        
        #2. para la columna MA recalculo el promedio y ya no es 0
        data_forecast['MA'] = data_forecast['quantity'].rolling(window=4).mean().shift(1)
        
        #2. reemplaza como valor real histórico el dado por data_set para calcular la siguiente predicción con ese dato
        data_forecast['quantity'].loc[len(data_forecast)-2] = data_test['quantity'].iloc[i]

    #Visualización
    
    #fig = go.Figure()
    #fig.add_trace(go.Scatter(x=data_training['semana'][:-1], y=data_training['quantity'], name="Training"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_test['quantity'], name="Test"))
    #fig.add_trace(go.Scatter(x=data_forecast['semana'][-len(data_test):], y=data_forecast['MA'][-len(data_test):], name="Pronóstico"))

    #fig.update_layout(title='Pronóstico según Promedio móvil simple',
    #                   xaxis_title='Semana',
    #                   yaxis_title='Demanda (unidades)')

    #fig.write_image("images/fig_1.png")
    #fig.show()

    return [data_test['quantity'], data_forecast['MA'][-len(data_test):]]

In [25]:
promedio_movil_simple(dicc_branch4, 0)

[68     1
 69     0
 70     1
 71     0
 72     0
 73     0
 74     0
 75     0
 76     1
 77     0
 78     0
 79     0
 80     0
 81     0
 82     0
 83     2
 84     0
 85     3
 86     0
 87     1
 88     0
 89     0
 90     0
 91     1
 92     0
 93     0
 94     0
 95     0
 96     0
 97     0
 98     0
 99     0
 100    0
 101    1
 Name: quantity, dtype: int64,
 103    0.50
 104    0.75
 105    0.75
 106    0.75
 107    0.75
 108    0.25
 109    0.25
 110    0.25
 111    0.25
 112    0.00
 113    0.00
 114    0.00
 115    0.00
 116    0.25
 117    0.25
 118    0.25
 119    0.25
 120    0.00
 121    0.00
 122    0.00
 123    0.00
 124    0.00
 125    0.00
 126    0.00
 127    0.00
 128    0.00
 129    0.00
 130    0.00
 131    0.00
 132    0.00
 133    0.00
 134    0.00
 135    0.00
 136    0.00
 Name: MA, dtype: float64]

In [26]:
promedio_movil_simple(dicc_branch4, 2)

[317    0
 318    0
 319    0
 320    0
 321    4
       ..
 384    0
 385    2
 386    0
 387    1
 388    2
 Name: quantity, Length: 72, dtype: int64,
 272    0.75
 273    0.50
 274    0.00
 275    0.00
 276    0.00
        ... 
 167    0.25
 168    1.00
 169    1.00
 170    0.75
 171    1.00
 Name: MA, Length: 72, dtype: float64]

In [27]:
# ------ FUNCIÓN MÉTODO SUAVIZACIÓN EXPONENCIAL SIMPLE ------

from statsmodels.tsa.api import SimpleExpSmoothing
import datetime
import math


datos_ses_branches = pd.DataFrame()
datos_ses_branches['branch'] = None
datos_ses_branches['item'] = None
datos_ses_branches['mae'] = None

def suavizacion_exp_simp(dicc_branch, item_key):
    
    data = dicc_branch[item_key]
    item = data['item_id'].iloc[0]
    
    data2 = data.copy()
    data = data2.drop(columns=['item_id'])
    
    #datos: [semana, quantity]

    data_training = data[:math.floor(int((len(data)*2/3)))+1]
    data_test = data[-math.ceil((int(len(data)*1/3))):]
    
    data_forecast = pd.DataFrame()
    data_forecast['test'] = None
    data_forecast['SES 1'] = None
    data_forecast['SES 2'] = None
    
    #smoothing_level: valor no definido aun con relación a datos 
    #ses_model = SimpleExpSmoothing(np.asarray(data_training['quantity'])).fit(smoothing_level=0.3, optimized=False)
    
    #primer pronóstico
    #data_forecast['SES'].loc[len(data_forecast)-1] = ses_model.forecast(1)
    #data_forecast['test'].loc[len(data_forecast)-1] = data_test.iloc[0][1]
    
    #actualizar el data con el que se hace el siguiente pronóstico. debe incluir el dato de test
    #data_training.loc[len(data_training)] = [data_test.iloc[0][0], data_test.iloc[0][1]]
    
    for i in range(0, len(data_test)):
        ses_model = SimpleExpSmoothing(np.asarray(data_training['quantity'])).fit(smoothing_level=0.3, optimized=False)
        forecast = ses_model.forecast(2)
        data_forecast.loc[len(data_forecast)] = [data_test.iloc[i][1], forecast[0], forecast[1]]

        #actualizar el data con el que se hace el siguiente pronóstico. debe incluir el dato de test
        data_training.loc[len(data_training)] = [data_test.iloc[i][0], data_test.iloc[i][1]]

    #fig = go.Figure()
    #fig.add_trace(go.Scatter(x=data_training['semana'], y=data_training['quantity'], name="Training"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_test['quantity'], name="Test"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_forecast['SES'], name="Pronóstico"))

    #fig.update_layout(title='Pronóstico según Suavización Exponencial Simple',
    #                   xaxis_title='Semana',
    #                   yaxis_title='Demanda (unidades)')

    #fig.show()
    
    return [data_test['quantity'], data_forecast['SES 1']]

In [28]:
suavizacion_exp_simp(dicc_branch4, 2)

[317    0
 318    0
 319    0
 320    0
 321    4
       ..
 384    0
 385    2
 386    0
 387    1
 388    2
 Name: quantity, Length: 72, dtype: int64,
 0     0.676196
 1     0.473337
 2     0.331336
 3     0.231935
 4     0.162355
         ...   
 67    0.210752
 68    0.210752
 69    0.210752
 70    0.210752
 71    0.210752
 Name: SES 1, Length: 72, dtype: float64]

In [29]:
# ----- FUNCIÓN SUAVIZACIÓN EXPONENCIAL DOBLE (Método de Holt) -----


from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import datetime
import math


datos_sed_branches = pd.DataFrame()
datos_sed_branches['branch'] = None
datos_sed_branches['item'] = None
datos_sed_branches['mae'] = None

def suavizacion_exp_doble(dicc_branch, item_key):
    
    data = dicc_branch[item_key]
    item = data['item_id'].iloc[0]
    
    data2 = data.copy()
    data = data2.drop(columns=['item_id'])
    
    #datos: [semana, quantity]

    data_training = data[:math.floor(int((len(data)*2/3)))+1]
    data_test = data[-math.ceil((int(len(data)*1/3))):]
    
    data_forecast = pd.DataFrame()
    data_forecast['test'] = None
    data_forecast['SED 1'] = None
    data_forecast['SED 2'] = None
    
    for i in range(0, len(data_test)):
        sed_model = Holt(np.asarray(data_training['quantity'])).fit(smoothing_level = 0.3, smoothing_slope = 0.2)
        forecast = sed_model.forecast(2)
        data_forecast.loc[len(data_forecast)] = [data_test.iloc[i][1], forecast[0], forecast[1]]

        #actualizar el data con el que se hace el siguiente pronóstico. debe incluir el dato de test
        data_training.loc[len(data_training)] = [data_test.iloc[i][0], data_test.iloc[i][1]]

    #fig = go.Figure()
    #fig.add_trace(go.Scatter(x=data_training['semana'], y=data_training['quantity'], name="Training"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_test['quantity'], name="Test"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_forecast['SES'], name="Pronóstico"))

    #fig.update_layout(title='Pronóstico según Suavización Exponencial Simple',
    #                   xaxis_title='Semana',
    #                   yaxis_title='Demanda (unidades)')

    #fig.show()
    
    return [data_test['quantity'], data_forecast['SED 1']]

In [30]:
# ----- FUNCIÓN SUAVIZACIÓN EXPONENCIAL TRIPLE (Método Holt-Winters)

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import datetime
import math


datos_set_branches = pd.DataFrame()
datos_set_branches['branch'] = None
datos_set_branches['item'] = None
datos_set_branches['mae'] = None

def suavizacion_exp_triple(dicc_branch, item_key):
    
    data = dicc_branch[item_key]
    item = data['item_id'].iloc[0]
    
    data2 = data.copy()
    data = data2.drop(columns=['item_id'])
    
    #datos: [semana, quantity]

    data_training = data[:math.floor(int((len(data)*2/3)))+1]
    data_test = data[-math.ceil((int(len(data)*1/3))):]
    
    data_forecast = pd.DataFrame()
    data_forecast['test'] = None
    data_forecast['SET 1'] = None
    data_forecast['SET 2'] = None
    
    for i in range(0, len(data_test)):
        
        set_model = ExponentialSmoothing(np.asarray(data_training['quantity']) ,seasonal_periods=2 ,trend='add', seasonal='add',).fit()
        forecast = set_model.forecast(2)
        data_forecast.loc[len(data_forecast)] = [data_test.iloc[i][1], forecast[0], forecast[1]]

        #actualizar el data con el que se hace el siguiente pronóstico. debe incluir el dato de test
        data_training.loc[len(data_training)] = [data_test.iloc[i][0], data_test.iloc[i][1]]

    #fig = go.Figure()
    #fig.add_trace(go.Scatter(x=data_training['semana'], y=data_training['quantity'], name="Training"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_test['quantity'], name="Test"))
    #fig.add_trace(go.Scatter(x=data_test['semana'], y=data_forecast['SES'], name="Pronóstico"))

    #fig.update_layout(title='Pronóstico según Suavización Exponencial Simple',
    #                   xaxis_title='Semana',
    #                   yaxis_title='Demanda (unidades)')

    #fig.show()
    
    return [data_test['quantity'], data_forecast['SET 1']]

In [31]:
# ----- MÉTODO SARIMA (ajuste de parámetros es automático) -----

from pmdarima import auto_arima
import datetime
import math

datos_ari_branches = pd.DataFrame()
datos_ari_branches['branch'] = None
datos_ari_branches['item'] = None
datos_ari_branches['mae'] = None

def sarima(dicc_branch, item_key):
    
    data = dicc_branch[item_key]
    item = data['item_id'].iloc[0]
    
    data2 = data.copy()
    data = data2.drop(columns=['item_id'])
    
    #datos: [semana, quantity]

    data_training = data[:math.floor(int((len(data)*2/3)))+1]
    data_test = data[-math.ceil((int(len(data)*1/3))):]
    
    data_forecast = pd.DataFrame()
    data_forecast['test'] = None
    data_forecast['sarima 1'] = None
    data_forecast['sarima 2'] = None

    for i in range(0, len(data_test)):
        
        model = auto_arima(y=np.asarray(data_training['quantity']), m=6)
        forecast = pd.Series(model.predict(n_periods = 2))
        data_forecast.loc[len(data_forecast)] = [data_test.iloc[i][1], forecast[0], forecast[1]]

        #actualizar el data con el que se hace el siguiente pronóstico. debe incluir el dato de test
        data_training.loc[len(data_training)] = [data_test.iloc[i][0], data_test.iloc[i][1]]

    return [data_test['quantity'], data_forecast['sarima 1']]
        

In [32]:
dicc_branch4

{0:      item_id  semana  quantity
 0          2     108         1
 1          2     109         0
 2          2     110         0
 3          2     111         0
 4          2     112         0
 ..       ...     ...       ...
 97         2     205         0
 98         2     206         0
 99         2     207         0
 100        2     208         0
 101        2     209         1
 
 [102 rows x 3 columns],
 1:      item_id  semana  quantity
 102        3     124         1
 103        3     125         1
 104        3     126         0
 105        3     127         0
 106        3     128         1
 ..       ...     ...       ...
 167        3     189         0
 168        3     190         1
 169        3     191         0
 170        3     192         0
 171        3     193         1
 
 [70 rows x 3 columns],
 2:      item_id  semana  quantity
 172       13       2         1
 173       13       3         0
 174       13       4         0
 175       13       5         0
 176      

In [33]:
datos_branches = pd.DataFrame()
datos_branches['branch'] = None
datos_branches['item'] = None
datos_branches['kpi pms'] = None
datos_branches['kpi ses'] = None
datos_branches['kpi sed'] = None
datos_branches['kpi set'] = None
datos_branches['kpi sar'] = None

In [34]:
for item in range(len(dicc_branch4)):
    print(item)
        
    try:
        print("r1")
        r1 = promedio_movil_simple(dicc_branch4, item)
        mae1 = mae(r1[0], r1[1])
    except ValueError:
        r1 = "error"
        mae1 = "error"
    try:
        print("r2")
        r2 = suavizacion_exp_simp(dicc_branch4, item)
        mae2 = mae(r2[0], r2[1])
    except ValueError:
        r2 = "error"
        mae2 = "error"
    try:
        print("r3")
        r3 = suavizacion_exp_doble(dicc_branch4, item)
        mae3 = mae(r3[0], r3[1])
    except ValueError: 
        r3 = "error"
        mae3 = "error"
    try:
        print("r4")
        r4 = suavizacion_exp_triple(dicc_branch4, item)
        mae4 = mae(r4[0], r4[1])
    except ValueError:
        r4 = "error"
        mae4 = "error"
    try:
        print("r5")
        r5 = sarima(dicc_branch4, item)
        mae5 = mae(r5[0], r5[1])
    except ValueError:
        r5 = "error"
        mae5 = "error"


    datos_branches.loc[len(datos_branches)] = [0,
                                               dicc_branch4[item]['item_id'].iloc[0],
                                               mae1,
                                               mae2,
                                               mae3,
                                               mae4,
                                               mae5]


0
r1
r2
r3
r4
r5


KeyboardInterrupt: 

In [ ]:
 datos_branches

,branch,item,mae pms,mae ses,mae sed,mae set,mae sar,total semanas con datos,diferencia
0,0.0,121.0,0.041667,0.056427,0.059349,0.191511,0.289924,22.0,72.0
1,0.0,192.0,0.347222,0.395554,0.428790,0.361586,0.443858,10.0,53.0
2,0.0,254.0,0.210526,0.297726,0.328636,0.350435,0.297347,10.0,56.0
3,0.0,365.0,0.833333,0.989396,1.045716,0.868313,0.83351,30.0,81.0
4,0.0,380.0,0.203704,0.293089,0.329988,0.330760,0.461794,23.0,80.0
5,0.0,416.0,0.181818,0.289141,0.323549,0.308735,0.494942,17.0,66.0
6,0.0,558.0,0.052632,0.052700,0.056014,0.202775,0.059336,9.0,58.0
7,0.0,561.0,0.555288,0.647846,0.697608,0.648866,0.593404,35.0,78.0
8,0.0,628.0,0.837963,0.901626,1.072431,0.943225,1.078707,21.0,80.0
9,0.0,641.0,0.052632,0.052637,0.054056,0.094934,0.130331,8.0,58.0


In [ ]:
datos_branches.to_excel('kpi_pronóstico_branch4.xlsx')

In [ ]:
mejor_metodo_branch4 = pd.DataFrame()
mejor_metodo_branch4['item'] = None
mejor_metodo_branch4['metodo'] = None

In [ ]:
import numpy as np 

for i in range(len(datos_branches)):
    minn = [5, ""]
    
    if type(datos_branches.loc[i][2]) == float or type(datos_branches.loc[i][2]) == np.float64:
        if datos_branches.loc[i][2] < minn[0]:
            minn = [datos_branches.loc[i][2], "pms"]
    
    if type(datos_branches.loc[i][3]) == float or type(datos_branches.loc[i][3]) == np.float64:
        if datos_branches.loc[i][3] < minn[0]:
            minn = [datos_branches.loc[i][3], "ses"]
    
    if type(datos_branches.loc[i][4]) == float or type(datos_branches.loc[i][4]) == np.float64:
        if datos_branches.loc[i][4] < minn[0]:
            minn = [datos_branches.loc[i][4], "sed"]
        
    if type(datos_branches.loc[i][5]) == float or type(datos_branches.loc[i][5]) == np.float64:
        if datos_branches.loc[i][5] < minn[0]:
            minn = [datos_branches.loc[i][5], "set"]
    
    if type(datos_branches.loc[i][6]) == float or type(datos_branches.loc[i][6]) == np.float64:
        if datos_branches.loc[i][6] < minn[0]:
            minn = [datos_branches.loc[i][6], "sar"]

    mejor_metodo_branch4.loc[len(mejor_metodo_branch4)] = [datos_branches.loc[i][1], minn[1]]

In [ ]:
mejor_metodo_branch4

,item,metodo
0,121.0,pms
1,192.0,pms
2,254.0,pms
3,365.0,pms
4,380.0,pms
5,416.0,pms
6,558.0,pms
7,561.0,pms
8,628.0,pms
9,641.0,pms


In [ ]:
mejor_metodo_branch4.to_excel('pronostico_demanda/metodos_branches/mejor_metodo_branch4.xlsx')